<a href="https://colab.research.google.com/github/oluwadamilola/Topic-Modelling/blob/main/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Prep work: Downloading necessary files
Before we get started, we need to download all of the data we'll be using.

singnal financial news dataset -

#import Libraries

In [2]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 34.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.1 which is incompatible.
tensorflow 2.12.0 

In [1]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# The Data Preprocessing

*   cnbc data was used because there is a limited resource on colab to handle large data




In [2]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/cnbc_headlines.csv")

# Clean it up a little bit, removing non-word characters (numbers and ___ etc)
#df.content = df.content.str.replace("[^A-Za-z ]", " ")

df.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Headlines,Time,Description
0,Jim Cramer: A better way to invest in the Covi...,"7:51 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer recommended buying..."
1,Cramer's lightning round: I would own Teradyne,"7:33 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer rings the lightnin..."
2,NaN,NaN,NaN
3,"Cramer's week ahead: Big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020","""We'll pay more for the earnings of the non-Co..."
4,IQ Capital CEO Keith Bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020","Keith Bliss, IQ Capital CEO, joins ""Closing Be..."


In [3]:
df.Headlines = df.Headlines.str.replace("[^A-Za-z ]", " ")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df.Headlines = df.Description.str.replace("[^A-Za-z ]", " ")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
df.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Headlines,Time,Description
0,"""Mad Money"" host Jim Cramer recommended buying...","7:51 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer recommended buying..."
1,"""Mad Money"" host Jim Cramer rings the lightnin...","7:33 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer rings the lightnin..."
2,NaN,NaN,NaN


In [6]:
#remove NAN from the data
data = df.dropna()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
data.head(2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Headlines,Time,Description
0,"""Mad Money"" host Jim Cramer recommended buying...","7:51 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer recommended buying..."
1,"""Mad Money"" host Jim Cramer rings the lightnin...","7:33 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer rings the lightnin..."


#Text preprocessing


*   most of the topics are in content and title
*   



In [8]:


data = data.Description

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Tokenize words and cleanup the text

In [9]:
import nltk
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:

import nltk
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words("english"))

tokenized_documents = []
for document in data:
    tokens = word_tokenize(document.lower())
    filtered_tokens = [token for token in tokens if token not in stop_words]
    tokenized_documents.append(filtered_tokens)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# Tokenize the text data
tokenized_data = [doc.lower().split() for doc in data]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
# Create a dictionary from the tokenized data
dictionary = corpora.Dictionary(tokenized_documents)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#create Dictionary

In [14]:
# Convert the tokenized data into a document-term matrix
corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
corpus = [dictionary.doc2bow(tokens) for tokens in tokenized_documents]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
print(corpus )

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(0, 1), (1, 1), (2, 1), (5, 1), (8, 1), (9, 1), (10, 1), (11, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)], [(0, 1), (1, 1), (2, 1), (4, 1), (5, 1), (14, 1), (16, 1), (17, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)], [(0, 1), (1, 1), (2, 1), (17, 3), (19, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1)], [(0, 2), (1, 1), (2, 2), (4, 1), (8, 1), (10, 1), (11, 1), (17, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1)], [(0, 1), (1, 1), (2, 1), (5, 1), (8, 1), (9, 1), (10, 1), (11, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Build LDA Model

In [17]:
# Build the LDA model
from gensim import models
lda_model = gensim.models.LdaModel(
    corpus,
    num_topics=20,
    id2word=dictionary,
    passes=10,
    random_state=42
)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Print the topics and their top words
for topic_id, topic_words in lda_model.print_topics():
    print("Topic", topic_id, ":", topic_words)

Topic 0 : 0.083*"&" + 0.069*"p" + 0.050*"500" + 0.045*"." + 0.033*"," + 0.026*"%" + 0.017*"percent" + 0.014*"month" + 0.013*"year" + 0.013*"pain"
Topic 1 : 0.075*"cramer" + 0.075*"jim" + 0.073*"." + 0.033*"earnings" + 0.032*"'s" + 0.024*"week" + 0.020*"ahead" + 0.019*"stocks" + 0.014*"explains" + 0.013*"company"
Topic 2 : 0.030*"cramer" + 0.029*"jim" + 0.021*"." + 0.020*"inspect" + 0.017*"r" + 0.016*"toys" + 0.015*"understand" + 0.015*"taking" + 0.014*"us" + 0.014*"cryptocurrency"
Topic 3 : 0.072*"." + 0.046*"cramer" + 0.046*"jim" + 0.042*"," + 0.021*"'s" + 0.017*"market" + 0.017*"says" + 0.010*"year" + 0.010*"sits" + 0.009*"stocks"
Topic 4 : 0.079*"," + 0.057*"``" + 0.057*"''" + 0.056*"." + 0.030*"'s" + 0.023*"says" + 0.018*"cramer" + 0.018*"money" + 0.018*"jim" + 0.017*"said"
Topic 5 : 0.043*"money" + 0.042*"mad" + 0.042*"stock" + 0.041*"''" + 0.041*"``" + 0.040*"round" + 0.039*"host" + 0.039*"rapid" + 0.039*"'s" + 0.039*"means"
Topic 6 : 0.047*"." + 0.039*"street" + 0.038*"wall" + 0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
# Get the topic distribution for a specific document
document = tokenized_documents[0]
document_bow = dictionary.doc2bow(document)
topic_distribution = lda_model.get_document_topics(document_bow)
print("Topic Distribution for Document 1:", topic_distribution)

Topic Distribution for Document 1: [(5, 0.3842613), (9, 0.4648071), (13, 0.09777612)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Visualize the Topics

In [20]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.093789  0.133166       1        1  21.945791
18     0.100854  0.073885       2        1   9.632113
17     0.152342 -0.112512       3        1   9.004529
5      0.148019  0.270304       4        1   8.366638
16     0.216208  0.091450       5        1   8.213182
3      0.117448 -0.070698       6        1   6.820595
12     0.147025 -0.115380       7        1   6.315466
1      0.145416 -0.124280       8        1   5.895732
7      0.086843  0.047637       9        1   5.065844
9      0.035358 -0.059971      10        1   3.370067
0     -0.067108  0.037745      11        1   2.997128
6     -0.010366 -0.045781      12        1   2.797621
8      0.024046 -0.079230      13        1   2.316399
13     0.027335 -0.075329      14        1   2.132712
19    -0.069261 -0.025733      15        1   1.935028
2     -0.119110 -0.020910      16        1   1.274851
14    -0.262956  0.019997      17        1   0.594254
10    -0.258226  0.021969      18        1   0.551730
15    -0.258200  0.020547      19        1   0.426178
11    -0.249458  0.013123      20        1   0.344141, topic_info=           Term         Freq        Total Category  logprob  loglift
15            '   470.000000   470.000000  Default  30.0000  30.0000
0            ''   995.000000   995.000000  Default  29.0000  29.0000
2            ``   994.000000   994.000000  Default  28.0000  28.0000
9           jim  2108.000000  2108.000000  Default  27.0000  27.0000
5        cramer  2124.000000  2124.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
17            ,     0.025845  2053.870837  Topic20  -8.7979  -5.6112
442      sports     0.025827    11.585777  Topic20  -8.7985  -0.4342
2822        dry     0.025806     3.165187  Topic20  -8.7994   0.8625
1158      hopes     0.025804     2.402012  Topic20  -8.7995   1.1383
77    investing     0.025789    42.303094  Topic20  -8.8000  -1.7308

[1126 rows x 6 columns], token_table=      Topic      Freq Term
term                      
2895      2  0.946935    !
68        1  0.386514    $
68       12  0.193257    $
68       17  0.386514    $
136       7  0.024578    %
...     ...       ...  ...
4113     12  0.909692    –
783       1  0.585399    —
783       9  0.266091    —
783      10  0.053218    —
783      16  0.053218    —

[1799 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 19, 18, 6, 17, 4, 13, 2, 8, 10, 1, 7, 9, 14, 20, 3, 15, 11, 16, 12])

In [21]:
# Printing the topics and their corresponding keywords
for topic_id, topic_keywords in lda_model.print_topics():
    print(f"Topic ID: {topic_id}")
    print(f"Keywords: {topic_keywords}")
    print()

Topic ID: 0
Keywords: 0.083*"&" + 0.069*"p" + 0.050*"500" + 0.045*"." + 0.033*"," + 0.026*"%" + 0.017*"percent" + 0.014*"month" + 0.013*"year" + 0.013*"pain"

Topic ID: 1
Keywords: 0.075*"cramer" + 0.075*"jim" + 0.073*"." + 0.033*"earnings" + 0.032*"'s" + 0.024*"week" + 0.020*"ahead" + 0.019*"stocks" + 0.014*"explains" + 0.013*"company"

Topic ID: 2
Keywords: 0.030*"cramer" + 0.029*"jim" + 0.021*"." + 0.020*"inspect" + 0.017*"r" + 0.016*"toys" + 0.015*"understand" + 0.015*"taking" + 0.014*"us" + 0.014*"cryptocurrency"

Topic ID: 3
Keywords: 0.072*"." + 0.046*"cramer" + 0.046*"jim" + 0.042*"," + 0.021*"'s" + 0.017*"market" + 0.017*"says" + 0.010*"year" + 0.010*"sits" + 0.009*"stocks"

Topic ID: 4
Keywords: 0.079*"," + 0.057*"``" + 0.057*"''" + 0.056*"." + 0.030*"'s" + 0.023*"says" + 0.018*"cramer" + 0.018*"money" + 0.018*"jim" + 0.017*"said"

Topic ID: 5
Keywords: 0.043*"money" + 0.042*"mad" + 0.042*"stock" + 0.041*"''" + 0.041*"``" + 0.040*"round" + 0.039*"host" + 0.039*"rapid" + 0.039

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Topic Interpretation

In [30]:
# Print the topics and their top words
for topic_id, topic_words in lda_model.print_topics():
    print(f"Topic {topic_id}:")
    print(topic_words)
    print()

Topic 0:
0.083*"&" + 0.069*"p" + 0.050*"500" + 0.045*"." + 0.033*"," + 0.026*"%" + 0.017*"percent" + 0.014*"month" + 0.013*"year" + 0.013*"pain"

Topic 1:
0.075*"cramer" + 0.075*"jim" + 0.073*"." + 0.033*"earnings" + 0.032*"'s" + 0.024*"week" + 0.020*"ahead" + 0.019*"stocks" + 0.014*"explains" + 0.013*"company"

Topic 2:
0.030*"cramer" + 0.029*"jim" + 0.021*"." + 0.020*"inspect" + 0.017*"r" + 0.016*"toys" + 0.015*"understand" + 0.015*"taking" + 0.014*"us" + 0.014*"cryptocurrency"

Topic 3:
0.072*"." + 0.046*"cramer" + 0.046*"jim" + 0.042*"," + 0.021*"'s" + 0.017*"market" + 0.017*"says" + 0.010*"year" + 0.010*"sits" + 0.009*"stocks"

Topic 4:
0.079*"," + 0.057*"``" + 0.057*"''" + 0.056*"." + 0.030*"'s" + 0.023*"says" + 0.018*"cramer" + 0.018*"money" + 0.018*"jim" + 0.017*"said"

Topic 5:
0.043*"money" + 0.042*"mad" + 0.042*"stock" + 0.041*"''" + 0.041*"``" + 0.040*"round" + 0.039*"host" + 0.039*"rapid" + 0.039*"'s" + 0.039*"means"

Topic 6:
0.047*"." + 0.039*"street" + 0.038*"wall" + 0.

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Document Labelling
from the corpus of words

In [31]:
# Create a list to store the dominant topics for each document
dominant_topics = []

# Iterate over each document in the corpus
for document_bow in corpus:
    # Get the topic distribution for the document
    topic_distribution = lda_model.get_document_topics(document_bow)
    # Sort the topics by their probability in descending order
    sorted_topics = sorted(topic_distribution, key=lambda x: x[1], reverse=True)
    # Get the dominant topic (topic with the highest probability)
    dominant_topic = sorted_topics[0][0]
    dominant_topics.append(dominant_topic)

# Print the dominant topics for each document
for i, topic in enumerate(dominant_topics):
    print(f"Document {i+1}: Dominant Topic - {topic}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Document 1: Dominant Topic - 9
Document 2: Dominant Topic - 5
Document 3: Dominant Topic - 4
Document 4: Dominant Topic - 4
Document 5: Dominant Topic - 18
Document 6: Dominant Topic - 5
Document 7: Dominant Topic - 18
Document 8: Dominant Topic - 3
Document 9: Dominant Topic - 4
Document 10: Dominant Topic - 1
Document 11: Dominant Topic - 5
Document 12: Dominant Topic - 4
Document 13: Dominant Topic - 5
Document 14: Dominant Topic - 4
Document 15: Dominant Topic - 12
Document 16: Dominant Topic - 4
Document 17: Dominant Topic - 4
Document 18: Dominant Topic - 3
Document 19: Dominant Topic - 5
Document 20: Dominant Topic - 4
Document 21: Dominant Topic - 9
Document 22: Dominant Topic - 3
Document 23: Dominant Topic - 6
Document 24: Dominant Topic - 4
Document 25: Dominant Topic - 5
Document 26: Dominant Topic - 7
Document 27: Dominant Topic - 4
Document 28: Dominant Topic - 5
Document 29: Dominant Topic - 4
Document 30: Dominant Topic - 4
Document 31: Dominant Topic - 5
Document 32: D

# implementing Topic Coherence Evaluation
To perform topic coherence evaluation on the topics generated by the topic modeling algorithm

In [38]:
from gensim.models import CoherenceModel
# Compute topic coherence using the UMass measure
coherence_model_umass = CoherenceModel(model=lda_model, texts=tokenized_documents, dictionary=dictionary, coherence='u_mass')
coherence_umass = coherence_model_umass.get_coherence()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
# Compute topic coherence using the C_v measure
coherence_model_cv = CoherenceModel(model=lda_model, texts=tokenized_documents, dictionary=dictionary, coherence='c_v')
coherence_cv = coherence_model_cv.get_coherence()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
# Compute topic coherence using the C_uci measure
coherence_model_uci = CoherenceModel(model=lda_model, texts=tokenized_documents, dictionary=dictionary, coherence='c_uci')
coherence_uci = coherence_model_uci.get_coherence()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
# Compute topic coherence using the C_npmi measure
coherence_model_npmi = CoherenceModel(model=lda_model, texts=tokenized_documents, dictionary=dictionary, coherence='c_npmi')
coherence_npmi = coherence_model_npmi.get_coherence()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
# Print the coherence scores
print(f"Topic Coherence (UMass): {coherence_umass}")
print(f"Topic Coherence (C_v): {coherence_cv}")
print(f"Topic Coherence (C_uci): {coherence_uci}")
print(f"Topic Coherence (C_npmi): {coherence_npmi}")

Topic Coherence (UMass): -7.832619347169372
Topic Coherence (C_v): 0.36523121467023206
Topic Coherence (C_uci): -4.238843461525923
Topic Coherence (C_npmi): -0.11912885490914193


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
